<a href="https://colab.research.google.com/github/timthedev07/news-articles-classification/blob/dev/news_articles_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
import os
import string
import re
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
stopWords = set(stopwords.words("english"))
stopWords.remove("not")

Load data.

In [ ]:
!rm -rf ./sample_data
!rm -rf ./data
!mkdir data

from google.colab import drive
drive.mount('/content/gdrive')

!cp "/content/gdrive/My Drive/datasets/news-articles-categorization/data.json.zip" data
%cd data
!unzip -ojq data.json.zip
!mv News_Category_Dataset_v2.json data.json
!rm -rf data.json.zip
%cd ..

data = pd.read_json("data/data.json", lines = True)

In [ ]:
x = data["headline"]
y = pd.get_dummies(data["category"])

trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.2, random_state = 42, shuffle = True)

LABELS = np.sort(data["category"].unique())

Input cleaning.

In [ ]:
def customStandardization(text: tf.Tensor):
    # to lower case
    text = tf.strings.lower(text)
    # expand contraction
    pairs = [
        ("won't", "will not"),
        ("can't", "can not"),
        ("n't", " not"),
        ("'re", " are"),
        ("'s", " is"),
        ("'d", " would"),
        ("'ll", " will"),
        ("'t", " not"),
        ("'ve", " have"),
        ("'m", " am"),
    ]
    for contracted, replacement in pairs:
        text = tf.strings.regex_replace(text, contracted, replacement)
    
    # clean special symbols
    text = tf.strings.regex_replace(text, r"\d+(?:\.\d*)?(?:[eE][+-]?\d+)?", " ")
    text = tf.strings.regex_replace(text, r'@([A-Za-z0-9_]+)', " ")
    text = tf.strings.regex_replace(text, r"[^A-Za-z0-9]+", " ")

    # remove stopwords
    for i in stopWords:
        text = tf.strings.regex_replace(text, f"[^A-Za-z0-9_]+{i}[^A-Za-z0-9_]+", " ")

    return text

The model.

In [ ]:
model = Sequential([
    
])